In [220]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [221]:
path_train = "UJIndoorLoc/trainingData.csv"
path_validation = "UJIndoorLoc/validationData.csv"

In [222]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.ix[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths))

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
#categorical => no intrinsic ordering 
dummy_labels = pd.get_dummies(train_labels)

"""
#corresponding column names
dummies = "00  01  02  03  10  11  12  13  20  21  22  23  24".split() 
"""

"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

In [223]:
#generate len(AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [224]:
# We will then split our given training set into training + validation 
train_X = AP_features[train_val_split]
train_y = labels[train_val_split]
val_X = AP_features[~train_val_split]
val_y = labels[~train_val_split]

In [225]:
train_df.shape

(19930, 529)

In [226]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.ix[:,0:520]))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [227]:
nb_epochs = 20
batch_size = 10
input_size = 520
num_classes = 13

In [228]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model

In [229]:
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
    

In [230]:
e = encoder()

In [231]:
d = decoder(e)

In [232]:
d.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_93 (Dense)                 (None, 256)           133376      dense_input_16[0][0]             
____________________________________________________________________________________________________
dense_94 (Dense)                 (None, 128)           32896       dense_93[0][0]                   
____________________________________________________________________________________________________
dense_95 (Dense)                 (None, 64)            8256        dense_94[0][0]                   
____________________________________________________________________________________________________
dense_96 (Dense)                 (None, 128)           8320        dense_95[0][0]                   
___________________________________________________________________________________________

In [233]:
d.fit(train_X, train_X, nb_epoch=nb_epochs, batch_size=batch_size)

Epoch 1/20
14035/14035 [==============================] - 12s - loss: 0.7133    
Epoch 2/20
14035/14035 [==============================] - 12s - loss: 0.6888    
Epoch 3/20
14035/14035 [==============================] - 12s - loss: 0.6830    
Epoch 4/20
14035/14035 [==============================] - 12s - loss: 0.6799    
Epoch 5/20
14035/14035 [==============================] - 12s - loss: 0.6777    
Epoch 6/20
14035/14035 [==============================] - 12s - loss: 0.6758    
Epoch 7/20
14035/14035 [==============================] - 12s - loss: 0.6744    
Epoch 8/20
14035/14035 [==============================] - 12s - loss: 0.6729    
Epoch 9/20
14035/14035 [==============================] - 12s - loss: 0.6717    
Epoch 10/20
14035/14035 [==============================] - 12s - loss: 0.6707    
Epoch 11/20
14035/14035 [==============================] - 13s - loss: 0.6697    
Epoch 12/20
14035/14035 [==============================] - 13s - loss: 0.6686    
Epoch 13/20
14035/14035 [

In [234]:
def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    d.add(Dense(128, activation='tanh', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d

In [235]:
c = classifier(d)

In [236]:
c.fit(train_X, train_y, validation_data=(val_X, val_y), nb_epoch=nb_epochs, batch_size=batch_size)


Train on 14035 samples, validate on 5902 samples
Epoch 1/20
14035/14035 [==============================] - 11s - loss: 0.3195 - acc: 0.8861 - val_loss: 0.2068 - val_acc: 0.9287
Epoch 2/20
14035/14035 [==============================] - 10s - loss: 0.1306 - acc: 0.9557 - val_loss: 0.1510 - val_acc: 0.9473
Epoch 3/20
14035/14035 [==============================] - 10s - loss: 0.1065 - acc: 0.9637 - val_loss: 0.1322 - val_acc: 0.9568
Epoch 4/20
14035/14035 [==============================] - 10s - loss: 0.0898 - acc: 0.9698 - val_loss: 0.1342 - val_acc: 0.9610
Epoch 5/20
14035/14035 [==============================] - 11s - loss: 0.0790 - acc: 0.9724 - val_loss: 0.1886 - val_acc: 0.9448
Epoch 6/20
14035/14035 [==============================] - 11s - loss: 0.0711 - acc: 0.9774 - val_loss: 0.1362 - val_acc: 0.9610
Epoch 7/20
14035/14035 [==============================] - 11s - loss: 0.0652 - acc: 0.9768 - val_loss: 0.1704 - val_acc: 0.9556
Epoch 8/20
14035/14035 [==============================]

In [244]:
loss, acc = c.evaluate(test_AP_features, test_labels)

1088/1111 [============================>.] - ETA: 0s

In [246]:
c.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_93 (Dense)                 (None, 256)           133376      dense_input_16[0][0]             
____________________________________________________________________________________________________
dense_94 (Dense)                 (None, 128)           32896       dense_93[0][0]                   
____________________________________________________________________________________________________
dense_95 (Dense)                 (None, 64)            8256        dense_94[0][0]                   
____________________________________________________________________________________________________
dense_99 (Dense)                 (None, 128)           8320        dense_95[0][0]                   
___________________________________________________________________________________________